<a href="https://colab.research.google.com/github/Harshkotkar/AI-agents-/blob/main/test_myntra_support_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install -U langchain langchain_core langchain_community langchain-text-splitters
!pip install pypdf
!pip install langchain_google_genai
!pip install bs4


In [46]:
!pip install langchain_chroma

In [47]:
# gemini setup
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter the api key ")

Enter the api key ··········


In [100]:
import bs4
from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationChain
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [49]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)


In [88]:
loder=WebBaseLoader(web_path="https://www.hindustantimes.com/cricket/pakistan-boycott-t20-world-cup-match-against-india-confirm-tournament-participation-101769945127505.html",
                    bs_kwargs=dict(
                        # parse_only=bs4.SoupStrainer(id="main-content")
                        parse_only=bs4.SoupStrainer("div", class_="articlePage")

                    ))



In [89]:
docs=loder.load()

In [90]:
# Re-run the splitting logic with the loaded docs
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split = text_spliter.split_documents(docs)

print(f"Number of chunks: {len(split)}")

Number of chunks: 6


In [91]:
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,
                                            chunk_overlap=200)
split=text_spliter.split_documents(docs)


In [92]:
split[:3]

[Document(metadata={'source': 'https://www.hindustantimes.com/cricket/pakistan-boycott-t20-world-cup-match-against-india-confirm-tournament-participation-101769945127505.html'}, page_content="Pakistan announces boycott of T20 World Cup match against India; confirms tournament participation, travel to Sri LankaPakistan confirm participation in the T20 World Cup, but the team won't take the field against India on February 15 in Colombo.\xa0Updated on: Feb 01, 2026 11:08 PM ISTWritten by Vishesh RoyShare viaCopy link Pakistan won't pull out of the upcoming T20 World Cup set to be played in India and Sri Lanka from February 7 to March 8, but the team won't be taking the field against arch-rival India on February 15 in Colombo. The decision was confirmed by the Pakistan government on Sunday evening, following a meeting between the country's Prime Minister Shehbaz Sharif and PCB chief Mohsin Naqvi. Ever since the International Cricket Council (ICC) replaced Bangladesh with Scotland in the 20

In [94]:
model_name="BAAI/bge-Base-en-v1.5"
model=HuggingFaceEmbeddings(model_name=model_name,
                            model_kwargs={"device":"cpu"},
                            encode_kwargs={"normalize_embeddings":True})

/tmp/ipython-input-2608610247.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  model=HuggingFaceEmbeddings(model_name=model_name,
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [97]:
db=Chroma.from_documents(documents=split,embedding=model)

In [99]:
retriver=db.as_retriever()

In [ ]:
prompt = PromptTemplate.from_template("""Answer  the question based only on the provided context answer as you are answering a childern of "12" years:

<context>
{context}
</context>

Question: {input}""")